# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 
import re

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt


def clean_non_western_encoded_characters_commands(text: str) -> str:
    """ Remove non-western encoded characters from a string
    List may need to grow.
    
    :param text: the text to clean
    :return: the cleaned text
    """
    text = re.sub(r"(\\begin{CJK}{UTF8}{gbsn})(.*?)(\\end{CJK})", r"\2", text)
    return text


def get_initials(name: str) -> str:
    """ Get the short name, e.g., A.-B. FamName
    :param name: full name
    :returns: initials
    """
    initials = []
    # account for non western names often in ()
    if '(' in name:
        name = clean_non_western_encoded_characters_commands(name)
        suffix = re.findall(r"\((.*?)\)", name)[0]
        name = name.replace(f"({suffix})", '')
    else:
        suffix = ''
    split = name.split()
    for token in split[:-1]:
        if '-' in token:
            current = '-'.join([k[0] + '.' for k in token.split('-')])
        else:
            current = token[0] + '.'
        initials.append(current)
    initials.append(split[-1].strip())
    if suffix:
        initials.append(f"({suffix})")
    return ' '.join(initials)

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# deal with the author list and edge cases of people that cannot be consistent on their name  

def filter_non_scientists(name: str) -> bool:
    """ Loose filter on expected authorships

    removing IT, administration, technical staff
    :param name: name
    :returns: False if name is not a scientist
    """
    remove_list = ['Licht', 'Binroth', 'Witzel', 'Jordan',
                   'Zähringer', 'Scheerer', 'Hoffmann', 'Düe',
                   'Hellmich', 'Enkler-Scharpegge', 'Witte-Nguy',
                   'Dehen', 'Beckmann', 'Jager', 'Jäger'
                  ]

    for k in remove_list:
        if k in name:
            return False
    return True

def add_author_to_list(author_list: list) -> list:
    """ Add author to list if not already in list
    
    :param author: author name
    :param author_list: list of authors
    :returns: updated list of authors
    """
    add_list = ['T. Henning']

    for author in add_list:
        if author not in author_list:
            author_list.append(author)
    return author_list

# get list from MPIA website
# filter for non-scientists (mpia.get_mpia_mitarbeiter_list() does some filtering)
mpia_authors = [k[1] for k in mpia.get_mpia_mitarbeiter_list() if filter_non_scientists(k[1])]
# add some missing author because of inconsistencies in their MPIA name and author name on papers
mpia_authors = add_author_to_list(mpia_authors)

In [4]:
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

def robust_call(fn, value, *args, **kwargs):
    try:
        return fn(value, *args, **kwargs)
    except Exception:
        return value

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [robust_call(mpia.get_initials, k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

X. Zhang  ->  X. Zhang  |  ['X. Zhang']
S. Shahaf  ->  S. Shahaf  |  ['S. Shahaf']
X. Zhang  ->  X. Zhang  |  ['X. Zhang']
Z. Fu  ->  Z. Fu  |  ['Z. Fu']
Arxiv has 55 new papers today
          4 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates):
    # debug crap
    paper['identifier'] = paper['identifier'].lower().replace('arxiv:', '').replace(r'\n', '').strip()
    paper_id = paper['identifier']
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [get_initials(k) for k in doc.authors], 
                mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print("Issues with the citations")
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/4 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2512.22963


extracting tarball to tmp_2512.22963... done.
Retrieving document from  https://arxiv.org/e-print/2512.23040


extracting tarball to tmp_2512.23040...

 done.


Issues with the citations
plugin pybtex.database.input.suffixes for suffix .tex not found
Retrieving document from  https://arxiv.org/e-print/2512.23060


extracting tarball to tmp_2512.23060... done.
Retrieving document from  https://arxiv.org/e-print/2512.23378
extracting tarball to tmp_2512.23378... done.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2512.23040-b31b1b.svg)](https://arxiv.org/abs/2512.23040) | **A Linearized Approach to Radial-Velocity Extraction. II: Shot-Noise-Limited Precision via Spectral Factorization**  |
|| <mark>S. Shahaf</mark>, B. Zackay |
|*Appeared on*| *2025-12-30*|
|*Comments*| *Submitted for publication in MNRAS*|
|**Abstract**|            We generalize the short-time Fourier transform (STFT) formalism for radial velocity extraction to cases where the underlying spectral components are unknown. The method factorizes a spectroscopic time series into principal spectra and time-dependent kernels, enabling simultaneous recovery of both. In Fourier space, each inverse-wavelength slice is decomposed by singular value decomposition, and radial velocity shifts are inferred from phase differences between epochs. In the high-SNR regime, this provides a linearized and statistically tractable estimate of differential velocities. The method is validated on synthetic and SOAP simulations and applied to EXPRES observations of HD 34411 and $\tau$ Ceti, recovering coherent signals and reaching the instrumental precision limit of ~30 cm/s. Apart from p-mode modulation, the residuals show no significant long-term correlations and allow the detection of signals with semi-amplitudes down to ~50 cm/s with $\lesssim10$ cm/s uncertainty. The framework thus enables extreme-precision radial velocity measurements in the presence of spectral variability, representing a step toward detecting and characterizing Earth-like planets around solar-type stars.         |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2512.22963-b31b1b.svg)](https://arxiv.org/abs/2512.22963) | **Impacts of the $^{16}$O($^{16}$O, n)$^{31}$S reaction rate on the evolution and nucleosynthesis in Pop III massive stars**  |
|| W. Xin, K. Nomoto, <mark>X. Zhang</mark>, S. Bi |
|*Appeared on*| *2025-12-30*|
|*Comments*| **|
|**Abstract**|            We first present a systematic investigation into the effect of the $^{16}$O($^{16}$O, n)$^{31}$S reaction rate on the evolution and nucleosynthesis of Population III (Pop III) stars. We simulate the evolution of a 15 M$_\odot$ Pop III star from the zero-age main sequence through to core collapse, while varying the $^{16}$O($^{16}$O, n)$^{31}$S reaction rate by factors of 0.1, 1, and 10. Our results demonstrate that increasing this reaction rate prompts earlier onset and extended duration of core oxygen burning at lower temperatures and densities. A higher reaction rate also increases neutron excess in OSi-rich layers, thereby promoting the synthesis of neutron-rich isotopes, particularly $^{31}$P and $^{39}$K. Most notably, the K yield is enhanced by a factor of 6.4. For a tenfold enhancement of the $^{16}$O($^{16}$O, n)$^{31}$S rate, the predicted [K/Ca] and [K/Fe] values from presupernova models reach 0.29 and 0.22 dex, respectively-values that are consistent with the most recent observational data for extremely metal-poor stars. These findings hold promise as a potential new solution to the problem of potassium underproduction and offer a valuable theoretical reference and motivation for subsequent measurements of oxygen fusion reaction rate.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2512.23060-b31b1b.svg)](https://arxiv.org/abs/2512.23060) | **Finding Quasars behind the Galactic Plane. IV. Candidate Selection from Chandra with Random Forest**  |
|| <mark>X. Zhang</mark>, et al. |
|*Appeared on*| *2025-12-30*|
|*Comments*| *ApJ, submitted*|
|**Abstract**|            Quasar samples remain severely incomplete at low Galactic latitudes because of strong extinction and source confusion. We conduct a systematic search for quasars behind the Galactic plane using X-ray sources from the Chandra Source Catalog (CSC 2.1), combined with optical data from Gaia DR3 and mid-infrared data from CatWISE2020. Using spectroscopically confirmed quasars and stars from data sets including DESI, SDSS, and LAMOST, we apply a Random Forest classifier to identify quasar candidates, with stellar contaminants suppressed using Gaia proper-motion constraints. Photometric redshifts are estimated for the candidates using a Random Forest regression model. Applying this framework to previously unclassified CSC sources, we identify 6286 quasar candidates, including 863 Galactic Plane Quasar (GPQ) candidates at |b|<20°, of which 514 are high-confidence candidates. Relative to the previously known GPQ sample, our selected GPQs reach fainter optical and X-ray fluxes, improving sensitivity to low-flux GPQs. In addition, both the GPQ candidates and known GPQs display harder X-ray spectra than the all-sky quasar sample, consistent with increased absorption through the Galactic plane. Pilot spectroscopy confirms two high-confidence GPQ candidates as quasars at spectroscopic redshifts of z=1.2582 and z=1.1313, and further spectroscopic follow-up of the GPQ sample is underway. This work substantially improves the census of GPQs and provides a valuable target sample for future spectroscopic follow-up, enabling the use of GPQs to refine the reference frames for astrometry and probe the Milky Way interstellar and circumgalactic media with the absorption features of GPQs.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2512.23378-b31b1b.svg)](https://arxiv.org/abs/2512.23378) | **Predicting the detection yields of giant planets and brown dwarfs with CSST astrometry**  |
|| Y. Xuan, et al. -- incl., <mark>Z. Fu</mark> |
|*Appeared on*| *2025-12-30*|
|*Comments*| *18 pages, 8 figures, 2 tables; accepted for publication in the Astronomical Journal*|
|**Abstract**|            Chinese Space Station Telescope (CSST), which will begin its scientific operations around 2027, is going to survey the sky area of the median-to-high Galactic latitude and median-to-high ecliptic latitude. The high astrometric precision of the CSST Survey Camera for faint objects enables the detection of a number of giant planets and brown dwarfs around M-dwarfs and brown dwarfs via differential astrometry in its optical survey. In this paper, we predict the number of giant planets and brown dwarfs around stars and brown dwarfs detectable with CSST astrometry. We generate synthetic samples of CSST stellar and substellar sources, and carry out companion injection-recovery simulations in the samples using different occurrence rates for FGK-dwarfs, M-dwarfs, and brown dwarfs. We calculate companion yields based on CSST astrometric precision. Our analysis reveals that over its 10-year mission, the CSST Survey Camera could barely discover giant planets and low-mass BDs around FGK-dwarfs, but is projected to detect 20 - 170 giant planets and low-mass brown dwarfs around M-dwarfs within 300 pc, and 300 - 570 brown dwarf binaries within 600 pc. Therefore, CSST astrometry is likely to significantly increase the current sample of substellar companions around M-dwarfs and brown dwarfs. This sample will deepen our understanding of planet formation and evolution around low-mass stars and brown dwarfs.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    print("found figures", fig_fnames)
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        if not os.path.exists(fname):
            print("file not found", fname)
            continue
        print("copying ", fname, "to", directory)
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

found figures ['tmp_2512.23040/./sv_phase_vs_zeta.png', 'tmp_2512.23040/./RV_extraction_SOAP.png', 'tmp_2512.23040/./U0_white_noise.png']
copying  tmp_2512.23040/./sv_phase_vs_zeta.png to _build/html/
copying  tmp_2512.23040/./RV_extraction_SOAP.png to _build/html/
copying  tmp_2512.23040/./U0_white_noise.png to _build/html/
exported in  _build/html/2512.23040.md
    + _build/html/tmp_2512.23040/./sv_phase_vs_zeta.png
    + _build/html/tmp_2512.23040/./RV_extraction_SOAP.png
    + _build/html/tmp_2512.23040/./U0_white_noise.png


## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\barak}[1]{{\color{blue} #1}}$
$\newcommand{\sahar}[1]{{\color{magenta} #1}}$
$\newcommand{\thebibliography}{\DeclareRobustCommand{\VAN}[3]{##3}\VANthebibliography}$</div>



<div id="title">

# A Linearized Approach to Radial-Velocity Extraction. II: Shot-Noise-Limited Precision via Spectral Factorization

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2512.23040-b31b1b.svg)](https://arxiv.org/abs/2512.23040)<mark>Appeared on: 2025-12-30</mark> -  _Submitted for publication in MNRAS_

</div>
<div id="authors">

<mark>S. Shahaf</mark>, B. Zackay

</div>
<div id="abstract">

**Abstract:** We generalize the short-time Fourier transform (STFT) formalism for radial velocity extraction to cases where the underlying spectral components are unknown. The method factorizes a spectroscopic time series into principal spectra and time-dependent kernels, enabling simultaneous recovery of both. In Fourier space, each inverse-wavelength slice is decomposed by singular value decomposition, and radial velocity shifts are inferred from phase differences between epochs. In the high-SNR regime, this provides a linearized and statistically tractable estimate of differential velocities. The method is validated on synthetic and SOAP simulations and applied to EXPRES observations of HD 34411 and $\tau$ Ceti, recovering coherent signals and reaching the instrumental precision limit of ${\sim}30$ cm s \textsuperscript{-1} . Apart from p-mode modulation, the residuals show no significant long-term correlations and allow the detection of signals with semi-amplitudes down to ${\sim}50$ cm s \textsuperscript{-1} with $\lesssim10$ cm s \textsuperscript{-1} uncertainty. The framework thus enables extreme-precision radial velocity measurements in the presence of spectral variability, representing a step toward detecting and characterizing Earth-like planets around solar-type stars.

</div>

<div id="div_fig1">

<img src="tmp_2512.23040/./sv_phase_vs_zeta.png" alt="Fig1" width="100%"/>

**Figure 1. -** _Top panel_---the normalized information in the first three principal spectra (corresponding to $\Sigma_{00}$, $\Sigma_{11}$ and $\Sigma_{22}$) versus their corresponding inverse-wavelength, $\zeta$, for the simulated white-noise spectroscopic time-series. All values are normalized to $\Sigma_{00}$ at $\zeta=0$.  _Bottom panel_---relative phases between the first and fourth simulated spectra, shown as a solid black line. The red stripes correspond to the expected relative phase due to the velocity difference between the two observations, i.e., $2\pi \zeta \times (v_4-v_1)$. Notably, the relative phase follows the expected trend for inverse wavelength below $\sim 0.2$ s km\textsuperscript{-1}, where the information content falls below the noise level. (*fig: singular values vs zeta*)

</div>
<div id="div_fig2">

<img src="tmp_2512.23040/./RV_extraction_SOAP.png" alt="Fig9" width="100%"/>

**Figure 9. -** _Top panels_---The rectified velocities, $v_{\rm c}$, extracted for the SOAP simulation described in Section \ref{sec: SOAP}. The simulated spectroscopic dataset is based on a synthetic spectrum of a Sun-like star, and the noise is assumed to be white and Gaussian. The injected radial velocity signal is a solid red curve.
        _Middle panels_---the extracted velocities from the zeroth-order kernel, $U^{(0)}$. These velocities show a combination of the injected Doppler shift and parasitic signal induced by stellar activity.
        _Bottom panels_---the extracted velocities from the first-order kernel, $U^{(1)}$, dominated by stellar activity. The solid red curve represents the fitted 5-harmonic model fitted to the extracted velocities (see text). (*fig: RV extraction soap*)

</div>
<div id="div_fig3">

<img src="tmp_2512.23040/./U0_white_noise.png" alt="Fig8" width="100%"/>

**Figure 8. -** _Top panel_---a selected segment from one of the simulated spectra used in Section \ref{sec: phase ratios}, centered around the MgB lines ($T_{\rm eff}=5{,}800   {\rm K}$; $\log g = 4.5$; ${\rm[Fe/H]}=0$; $v \sin i = 2  {\rm km  s}^{-1}$; and $\mathcal{R}=10^5$). _Bottom left panel_---the zeroth-order estimated of the principal kernels obtained from $\mathbf{U}_\zeta$, for the first seven spectra. The kernels are normalized such that their peak value is one and sorted one on top of the other. The arrows represent the Doppler shift used for each spectrum.
        _Bottom right panel_---same as the bottom-left panel, but using the reweighted matrices $\mathbf{U}_\zeta \Sigma_\zeta$. (*fig: U0 SVD*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2512.23040"></div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

130  publications files modified in the last 7 days.


In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

2  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [14]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

2  publications in the last day.


In [15]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
